todo list:
- Create a FeatureCollection of the different County Subdivision areas in the Georgia counties
listed in the R-acs.Rmd file 
- for impervious_descriptor layer:
    - keep values 20, 21, 22, 23, 25, 26, 27, 28, 29
    - take not of the mask 
- for the landcover layer, use the above as the mask 
- apply this for 
- keep this as some type of summary information for elsewhere
- do this for all the years (2001, 2004, 2006, 2008, 2011, 2013, 2016, 2019)


In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import ipyleaflet
import ipywidgets as widgets
from IPython.display import display

ee.Initialize()

In [2]:
def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi


def mask_classes(image, vals_to_keep):
    """Masks values of the image to only include those
    within vals_to_keep."""
    masks = []
    finalMask = ee.Image(0)

    for val in vals_to_keep:
        masks.append(image.eq(val))
    
    for mask in masks:
        finalMask = finalMask.Or(mask) 
    
    return image.updateMask(finalMask)


def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])


def get_labels(collection, class_name):
    """Returns a data frame containing the band values/class/corresponding
    palette color."""
    class_vals = collection.first().get(f"{class_name}_class_values").getInfo()
    class_labels = collection.first().get(f"{class_name}_class_names").getInfo()
    class_palette = collection.first().get(f"{class_name}_class_palette").getInfo()

    class_desc = [lab.split(':')[1] for lab in class_labels]
    class_labels = [lab.split(':')[0] for lab in class_labels]

    class_df = pd.DataFrame({
        'layer_vals': class_vals,
        'labels': class_labels,
        'class_description': class_desc,
        'palette': class_palette
    })

    return class_df


def get_legend_keys_values(class_df, vals_to_keep):
    legend_keys = list(class_df[class_df.layer_vals.isin(vals_to_keep)].labels)
    legend_keys = [leg.split(':')[0] for leg in legend_keys]
    legend_colors = list(class_df[class_df.layer_vals.isin(vals_to_keep)].palette)
    return legend_keys, legend_colors


def union_polygons(poly_ls):
    basePoly = poly_ls[0]
    for poly in poly_ls[1:]:
        basePoly = basePoly.union(poly, maxError = 1)

    return basePoly


def rect_from_corners(tl, br):
    return ee.Geometry.Polygon(
        [[tl[0], tl[1]],
        [br[0], tl[1]],
        [br[0], br[1]],
        [tl[0],br[1]]], None, False
    )

In [4]:
county_list = [
    'Cherokee', 'Clayton', 'Cobb', 
    'DeKalb', 'Douglas', 'Fayette', 
    'Forsyth', 'Fulton', 'Gwinnett', 
    'Henry','Rockdale'
]

ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
poly_list = []

for county in county_list:
    county_shp = ga_counties[ga_counties["NAME10"] == county].geometry.values[0]
    poly_list.append(pd_shp_to_ee_poly(county_shp).simplify(maxError = 1))

atlanta_roi = union_polygons(poly_list).simplify(maxError = 1)

ga_shp = gpd.read_file("ga-boundary/ga.shp").geometry[0]
ga_roi = pd_shp_to_ee_poly(ga_shp).simplify(maxError = 1)

In [14]:
collection = (
    ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
    .select('landcover')
    .map(lambda image: image.clip(atlanta_roi))
)

vals_to_keep = [21, 22, 23, 24]
years = [2001, 2004, 2006, 2008, 2011, 2013, 2016, 2019]

class_df = get_labels(collection, 'landcover')
legend_keys, legend_colors = get_legend_keys_values(class_df, vals_to_keep)

In [6]:
class_df

,layer_vals,labels,class_description,palette
0,11,Open water,"areas of open water, generally with less than...",466b9f
1,12,Perennial ice/snow,areas characterized by a perennial cover of i...,d1def8
2,21,"Developed, open space",areas with a mixture of some constructed mate...,dec5c5
3,22,"Developed, low intensity",areas with a mixture of constructed materials...,d99282
4,23,"Developed, medium intensity",areas with a mixture of constructed materials...,eb0000
5,24,Developed high intensity,highly developed areas where people reside or...,ab0000
6,31,Barren land (rock/sand/clay),"areas of bedrock, desert pavement, scarps, ta...",b3ac9f
7,41,Deciduous forest,areas dominated by trees generally greater th...,68ab5f
8,42,Evergreen forest,areas dominated by trees generally greater th...,1c5f2c
9,43,Mixed forest,areas dominated by trees generally greater th...,b5c58f


In [15]:
filtered_collection = collection.map(lambda image: mask_classes(image, vals_to_keep))
collection_list = filtered_collection.toList(filtered_collection.size())
#left_layer = geemap.ee_tile_layer(ee.Image(collection_list.get(0)), {}, '2001')
#right_layer = geemap.ee_tile_layer(ee.Image(collection_list.get(7)), {}, '2019')

In [16]:
Map = geemap.Map(height = "800px")
Map.setCenter(-83.42713, 32.63861, 8)
Map.addLayer(
    ee.Image(collection_list.get(1)), {}, "2004"
)
Map.addLayer(
    ee.Image(collection_list.get(3)), {}, "2008", shown = False
)
Map.addLayer(
    ee.Image(collection_list.get(5)), {}, "2013", shown = False
)
Map.addLayer(
    ee.Image(collection_list.get(7)), {}, "2019", shown = False
)
Map.add_legend(
    legend_keys = legend_keys, 
    legend_colors = legend_colors, 
    position = 'topright'
)
Map

Map(center=[32.63861, -83.42713], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…